<a href="https://colab.research.google.com/github/FiorelaCiroku/LLM---Akademia-e-Forcave-te-Armatosura/blob/main/Demo_Leksion_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip -q install faiss-cpu sentence-transformers rank-bm25 pandas numpy

In [ ]:
# Imports + toy “civil protection” corpus (with metadata)
# This includes the exact kind of synonym/paraphrase case from the slides (keyword fails, vector works).

import numpy as np
import pandas as pd
import time
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from rank_bm25 import BM25Okapi
import re
from dataclasses import dataclass, asdict

# A tiny corpus with metadata fields similar to the slides (source, language, year, authority)
docs = [
    {
        "doc_id": "SOP-AL-001",
        "title": "Procedura për njoftim përmbytjesh (Bashkia)",
        "text": "Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.",
        "source": "official",
        "authority": "municipality",
        "lang": "sq",
        "year": 2023,
    },
    {
        "doc_id": "SOP-AL-002",
        "title": "Protokoll Evakuimi",
        "text": "Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset.",
        "source": "official",
        "authority": "civil_protection",
        "lang": "sq",
        "year": 2022,
    },
    {
        "doc_id": "NATO-ENG-010",
        "title": "NATO Flood Response SOP",
        "text": "The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders.",
        "source": "official",
        "authority": "NATO",
        "lang": "en",
        "year": 2021,
    },
    {
        "doc_id": "BLOG-AL-777",
        "title": "Artikull jozyrtar",
        "text": "Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe mund të jetë i pasaktë.",
        "source": "unofficial",
        "authority": "blog",
        "lang": "sq",
        "year": 2019,
    },
    {
        "doc_id": "LAW-AL-100",
        "title": "Referencë ligjore",
        "text": "Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.",
        "source": "official",
        "authority": "law",
        "lang": "sq",
        "year": 2020,
    },
]

df = pd.DataFrame(docs)
df


,doc_id,title,text,source,authority,lang,year
0,SOP-AL-001,Procedura për njoftim përmbytjesh (Bashkia),"Në rast rreziku për përmbytje, Bashkia njofton...",official,municipality,sq,2023
1,SOP-AL-002,Protokoll Evakuimi,Evakuimi fillon kur Shërbimi i Emergjencave lë...,official,civil_protection,sq,2022
2,NATO-ENG-010,NATO Flood Response SOP,The incident commander issues warnings and coo...,official,NATO,en,2021
3,BLOG-AL-777,Artikull jozyrtar,Dikush mund të shkruajë online çfarëdo për për...,unofficial,blog,sq,2019
4,LAW-AL-100,Referencë ligjore,Neni 12: Autoriteti përgjegjës për menaxhimin ...,official,law,sq,2020


In [ ]:
# ---------
# Chunking
# ---------
import pandas as pd

def chunk_text(text, chunk_size=180, overlap=40):
    """
    Safe character-based chunker that:
    - never loops forever
    - validates parameters (overlap < chunk_size)
    - handles None/NaN safely
    """
    if text is None or (isinstance(text, float) and pd.isna(text)):
        return []
    text = str(text)

    if chunk_size <= 0:
        raise ValueError("chunk_size must be > 0")
    if overlap < 0:
        raise ValueError("overlap must be >= 0")
    if overlap >= chunk_size:
        raise ValueError("overlap must be < chunk_size (otherwise start won't advance)")

    chunks = []
    start = 0
    step = chunk_size - overlap  # guaranteed > 0

    while start < len(text):
        end = min(len(text), start + chunk_size)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        if end == len(text):
            break  # reached the end

        start += step  # always moves forward

    return chunks

# ----------------
# Build chunk table
# ----------------
rows = []
for _, r in df.iterrows():
    chunks = chunk_text(r["text"], chunk_size=160, overlap=30)
    for j, ch in enumerate(chunks):
        rows.append({
            "chunk_id": f"{r['doc_id']}::c{j}",
            "doc_id": r["doc_id"],
            "title": r["title"],
            "chunk_text": ch,
            "source": r["source"],
            "authority": r["authority"],
            "lang": r["lang"],
            "year": r["year"],
        })

chunks_df = pd.DataFrame(rows)
chunks_df


,chunk_id,doc_id,title,chunk_text,source,authority,lang,year
0,SOP-AL-001::c0,SOP-AL-001,Procedura për njoftim përmbytjesh (Bashkia),"Në rast rreziku për përmbytje, Bashkia njofton...",official,municipality,sq,2023
1,SOP-AL-002::c0,SOP-AL-002,Protokoll Evakuimi,Evakuimi fillon kur Shërbimi i Emergjencave lë...,official,civil_protection,sq,2022
2,NATO-ENG-010::c0,NATO-ENG-010,NATO Flood Response SOP,The incident commander issues warnings and coo...,official,NATO,en,2021
3,BLOG-AL-777::c0,BLOG-AL-777,Artikull jozyrtar,Dikush mund të shkruajë online çfarëdo për për...,unofficial,blog,sq,2019
4,LAW-AL-100::c0,LAW-AL-100,Referencë ligjore,Neni 12: Autoriteti përgjegjës për menaxhimin ...,official,law,sq,2020


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=chunks_df)

https://docs.google.com/spreadsheets/d/1sHWq0xcP9opWWkmbCTif1TQ0J9i5zQMgGFdNDa2lPyA/edit#gid=0


In [ ]:
# Embeddings model + vectorization
# Use a multilingual model so Albanian works well.

embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

emb = embed_model.encode(
    chunks_df["chunk_text"].tolist(),
    normalize_embeddings=True,   # cosine similarity via dot product
    show_progress_bar=True
).astype("float32")

emb.shape


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(5, 384)

In [ ]:
# Baseline: brute-force cosine search (slow at scale)
# This shows the “compare with all vectors” pattern.

def brute_force_search(query, k=5):
    q = embed_model.encode([query], normalize_embeddings=True).astype("float32")
    scores = (emb @ q[0])  # cosine since normalized
    top_idx = np.argsort(-scores)[:k]
    return [(int(i), float(scores[i])) for i in top_idx]

query = "Kush lëshon paralajmërime për përmbytje?"
t0 = time.time()
hits = brute_force_search(query, k=5)
t1 = time.time()

print("Brute-force time (ms):", round((t1-t0)*1000, 2))
for idx, s in hits:
    print(s, "->", chunks_df.iloc[idx][["chunk_id","doc_id","chunk_text"]].to_dict())


Brute-force time (ms): 90.51
0.6717923879623413 -> {'chunk_id': 'SOP-AL-001::c0', 'doc_id': 'SOP-AL-001', 'chunk_text': 'Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.'}
0.590835690498352 -> {'chunk_id': 'BLOG-AL-777::c0', 'doc_id': 'BLOG-AL-777', 'chunk_text': 'Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe mund të jetë i pasaktë.'}
0.5366114377975464 -> {'chunk_id': 'NATO-ENG-010::c0', 'doc_id': 'NATO-ENG-010', 'chunk_text': 'The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders.'}
0.4160260558128357 -> {'chunk_id': 'SOP-AL-002::c0', 'doc_id': 'SOP-AL-002', 'chunk_text': 'Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset.'}
0.28518086671829224 -> {'chunk_id': 'LAW-AL-100::c0', 'doc_id': 'LAW-AL-100', 'chunk_text': 'Neni 12: Autoriteti përgjegjës për menaxhimin

In [ ]:
# FAISS Vector DB: exact vs ANN (HNSW)
# This corresponds to the “indexing” section and ANN trade-off.

d = emb.shape[1]

# Exact index (Flat) for cosine (dot product on normalized vectors)
index_flat = faiss.IndexFlatIP(d)
index_flat.add(emb)

# ANN index (HNSW) - fast & robust in practice
index_hnsw = faiss.IndexHNSWFlat(d, 32)  # M=32
index_hnsw.hnsw.efSearch = 64
index_hnsw.add(emb)

def faiss_search(index, query, k=5):
    q = embed_model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(q, k)
    return list(zip(I[0].tolist(), D[0].tolist()))

query = "Kush lëshon paralajmërime për përmbytje?"

for name, idx in [("FLAT(exact)", index_flat), ("HNSW(ANN)", index_hnsw)]:
    t0 = time.time()
    hits = faiss_search(idx, query, k=5)
    t1 = time.time()
    print("\n==", name, "time(ms):", round((t1-t0)*1000, 2))
    for i, score in hits:
        row = chunks_df.iloc[i]
        print(round(score,4), row["chunk_id"], "|", row["chunk_text"])



== FLAT(exact) time(ms): 65.29
0.6718 SOP-AL-001::c0 | Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.
0.5908 BLOG-AL-777::c0 | Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe mund të jetë i pasaktë.
0.5366 NATO-ENG-010::c0 | The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders.
0.416 SOP-AL-002::c0 | Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset.
0.2852 LAW-AL-100::c0 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.

== HNSW(ANN) time(ms): 50.87
0.6564 SOP-AL-001::c0 | Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.
0.8183 BLOG-AL-777::c0 | Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe 

In [ ]:
# Metadata filtering (safety & precision)
# Filtering out outdated/unofficial sources.
# Demo approach: retrieve top-N from vector DB, then filter by metadata, then keep top-k.

def filtered_vector_search(query, top_n=20, k=5, filters=None, index=index_hnsw):
    filters = filters or {}
    raw = faiss_search(index, query, k=top_n)

    candidates = []
    for i, score in raw:
        r = chunks_df.iloc[i].to_dict()
        ok = True
        for key, val in filters.items():
            if isinstance(val, (list, tuple, set)):
                if r.get(key) not in val:
                    ok = False; break
            else:
                if r.get(key) != val:
                    ok = False; break
        if ok:
            candidates.append((i, float(score), r))

    candidates = sorted(candidates, key=lambda x: -x[1])[:k]
    return candidates

query = "Kush lëshon paralajmërime për përmbytje?"

print("Without filters:")
for i, s, r in filtered_vector_search(query, top_n=10, k=5, filters={}):
    print(round(s,4), r["chunk_id"], r["source"], r["year"], "|", r["chunk_text"])

print("\nWith filters (only official, year>=2020, lang sq/en):")
# simple filter: we’ll filter year via post-filter logic
cands = filtered_vector_search(query, top_n=20, k=20, filters={"source":"official"})
cands = [c for c in cands if c[2]["year"] >= 2020 and c[2]["lang"] in {"sq","en"}]
cands = sorted(cands, key=lambda x: -x[1])[:5]
for i, s, r in cands:
    print(round(s,4), r["chunk_id"], r["source"], r["year"], "|", r["chunk_text"])


Without filters:
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.

With filters (only official, year>=2020, lang sq/en):
3.4028234663852886e+38 LAW-AL-100::c0 official 2020 | Neni 12: Autoriteti përgjegjë

In [ ]:
# Hybrid search (BM25 keyword + vector)

def tokenize(s):
    s = s.lower()
    s = re.sub(r"[^a-zëç0-9\s]", " ", s)
    return [t for t in s.split() if t]

tokenized_corpus = [tokenize(x) for x in chunks_df["chunk_text"].tolist()]
bm25 = BM25Okapi(tokenized_corpus)

def bm25_search(query, k=5):
    scores = bm25.get_scores(tokenize(query))
    top = np.argsort(-scores)[:k]
    return [(int(i), float(scores[i])) for i in top]

def hybrid_search(query, k=5, w_vec=0.6, w_kw=0.4, top_n_vec=20, top_n_kw=20):
    # get candidate pool
    vec = faiss_search(index_hnsw, query, k=top_n_vec)
    kw  = bm25_search(query, k=top_n_kw)

    cand_ids = set([i for i,_ in vec] + [i for i,_ in kw])

    # normalize scores
    vec_dict = {i:s for i,s in vec}
    kw_dict  = {i:s for i,s in kw}

    vec_vals = np.array(list(vec_dict.values()) + [0.0], dtype="float32")
    kw_vals  = np.array(list(kw_dict.values()) + [0.0], dtype="float32")
    vmin, vmax = float(vec_vals.min()), float(vec_vals.max())
    kmin, kmax = float(kw_vals.min()), float(kw_vals.max())

    def norm(x, mn, mx):
        return 0.0 if mx == mn else (x - mn) / (mx - mn)

    scored = []
    for i in cand_ids:
        sv = norm(vec_dict.get(i, 0.0), vmin, vmax)
        sk = norm(kw_dict.get(i,  0.0), kmin, kmax)
        score = w_vec*sv + w_kw*sk
        scored.append((i, float(score), float(sv), float(sk)))

    scored = sorted(scored, key=lambda x: -x[1])[:k]
    return scored

# Query with an exact reference that keyword search should like
query_kw = "Neni 12"
print("BM25 only:")
for i, s in bm25_search(query_kw, k=5):
    print(round(s,4), chunks_df.iloc[i]["chunk_id"], "|", chunks_df.iloc[i]["chunk_text"])

print("\nHybrid:")
for i, s, sv, sk in hybrid_search(query_kw, k=5):
    print("hyb", round(s,4), "(vec", round(sv,3), "kw", round(sk,3), ")", chunks_df.iloc[i]["chunk_id"])


BM25 only:
2.2359 LAW-AL-100::c0 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.
0.0 SOP-AL-001::c0 | Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.
0.0 NATO-ENG-010::c0 | The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders.
0.0 SOP-AL-002::c0 | Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset.
0.0 BLOG-AL-777::c0 | Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe mund të jetë i pasaktë.

Hybrid:
hyb 0.6 (vec 1.0 kw 0.0 ) LAW-AL-100::c0
hyb 0.4 (vec 0.0 kw 1.0 ) LAW-AL-100::c0
hyb 0.0 (vec 0.0 kw 0.0 ) SOP-AL-002::c0
hyb 0.0 (vec 0.0 kw 0.0 ) NATO-ENG-010::c0
hyb 0.0 (vec 0.0 kw 0.0 ) SOP-AL-001::c0


In [ ]:
#Re-ranking (cross-encoder) on top-K

# Cross-encoder for reranking (pairwise scoring query, chunk)
# If you want multilingual reranking, try: "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1"
reranker = CrossEncoder("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")

def rerank(query, candidate_indices):
    pairs = [(query, chunks_df.iloc[i]["chunk_text"]) for i in candidate_indices]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(candidate_indices, scores.tolist()), key=lambda x: -x[1])
    return ranked

query = "Kush lëshon paralajmërime për përmbytje?"

# Step 1: fast ANN retrieve top-20
top_ann = faiss_search(index_hnsw, query, k=20)
cand_idx = [i for i,_ in top_ann]

# Step 2: rerank to top-5
reranked = rerank(query, cand_idx)[:5]

print("Top-5 after reranking:")
for i, s in reranked:
    row = chunks_df.iloc[i]
    print(round(s,4), row["chunk_id"], "|", row["chunk_text"])


Top-5 after reranking:
-1.6947 SOP-AL-002::c0 | Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset.
-1.8304 SOP-AL-001::c0 | Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit.
-3.3675 NATO-ENG-010::c0 | The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders.
-3.8705 BLOG-AL-777::c0 | Dikush mund të shkruajë online çfarëdo për përmbytjet; ky burim nuk është zyrtar dhe mund të jetë i pasaktë.
-8.1561 LAW-AL-100::c0 | Neni 12: Autoriteti përgjegjës për menaxhimin e emergjencave përcaktohet nga ligji dhe aktet nënligjore përkatëse.


In [ ]:
# Minimal logging (debuggable RAG retrieval)
# This follows the “what to log” points: query, returned chunks, similarity scores, filters, reranker model.

@dataclass
class RetrievalLog:
    query: str
    query_embedding_dim: int
    filters: dict
    ann_index: str
    top_n: int
    returned: list  # list of dicts
    reranker_model: str

def rag_retrieve(query, k=5, top_n=30, filters=None):
    filters = filters or {}
    # ANN retrieve
    raw = faiss_search(index_hnsw, query, k=top_n)

    # filter
    kept = []
    for i, score in raw:
        r = chunks_df.iloc[i].to_dict()
        ok = True
        for key, val in filters.items():
            if key == "year_gte":
                if r["year"] < val: ok = False
            elif r.get(key) != val:
                ok = False
        if ok:
            kept.append((i, float(score), r))

    # rerank
    cand_idx = [i for i,_,_ in kept[:min(len(kept), 50)]]
    reranked = rerank(query, cand_idx)[:k]

    returned = []
    for i, ce_score in reranked:
        r = chunks_df.iloc[i].to_dict()
        returned.append({
            "chunk_id": r["chunk_id"],
            "doc_id": r["doc_id"],
            "vector_score": next((s for j,s,_ in kept if j==i), None),
            "rerank_score": float(ce_score),
            "metadata": {k:r[k] for k in ["source","authority","lang","year"]},
            "text": r["chunk_text"]
        })

    log = RetrievalLog(
        query=query,
        query_embedding_dim=emb.shape[1],
        filters=filters,
        ann_index="FAISS:HNSWFlat",
        top_n=top_n,
        returned=returned,
        reranker_model="cross-encoder/mmarco-mMiniLMv2-L12-H384-v1"
    )
    return returned, log

results, log = rag_retrieve(
    "Kush lëshon paralajmërime për përmbytje?",
    k=3,
    top_n=25,
    filters={"source":"official", "year_gte": 2020}
)

print("RESULTS:\n")
for r in results:
    print(r["rerank_score"], r["chunk_id"], r["metadata"], "\n ", r["text"], "\n")

print("\nLOG (as dict):\n")
print(asdict(log))


RESULTS:

-1.6946672201156616 SOP-AL-002::c0 {'source': 'official', 'authority': 'civil_protection', 'lang': 'sq', 'year': 2022} 
  Evakuimi fillon kur Shërbimi i Emergjencave lëshon urdhër evakuimi dhe koordinohet me policinë dhe zjarrfikëset. 

-1.8304476737976074 SOP-AL-001::c0 {'source': 'official', 'authority': 'municipality', 'lang': 'sq', 'year': 2023} 
  Në rast rreziku për përmbytje, Bashkia njofton popullatën me paralajmërime zyrtare përmes kanaleve të komunikimit. 

-3.367546319961548 NATO-ENG-010::c0 {'source': 'official', 'authority': 'NATO', 'lang': 'en', 'year': 2021} 
  The incident commander issues warnings and coordinates evacuation with local authorities and emergency responders. 


LOG (as dict):

{'query': 'Kush lëshon paralajmërime për përmbytje?', 'query_embedding_dim': 384, 'filters': {'source': 'official', 'year_gte': 2020}, 'ann_index': 'FAISS:HNSWFlat', 'top_n': 25, 'returned': [{'chunk_id': 'SOP-AL-002::c0', 'doc_id': 'SOP-AL-002', 'vector_score': 1.16794776